# Load pretrained resNet to predict species of plant

### Import libraries and required packages

#### descriptions
1. **pandas** : reading csv file.(e.g., ground-truth)
2. **torch** : load a model for prediction.
3. **os** : find out your directory path.
4. **cv2** : generate a numeric representation from your orginal image file

In [1]:
import pandas as pd
import torch
import os
import cv2

### Define the directory

#### define the path your model stays

In [2]:
CKPT_DIR = 'ckpt'
CKPT_PATH = os.path.join(CKPT_DIR, 'resNet_15_compose.ckpt')

#### define the ground-truth path

In [3]:
LABEL_DIR = 'label2.csv'
label_df = pd.read_csv(LABEL_DIR)
label_df.head()

,dirpath,target,label
0,DaanForestPark/20180330/1lZsRrQzj,通泉草,66
1,DaanForestPark/20180330/4PdXwYcGt,紫花酢漿草,51
2,DaanForestPark/20180330/6VrmeiUE5,通泉草,66
3,DaanForestPark/20180330/7nrSoiuHL,通泉草,66
4,DaanForestPark/20180330/7RKdkJLgy,通泉草,66


#### define your img file

In [4]:
DATA_SAMPLE = 'sample/sample.jpg'

### Image to RGB representaion function

In [5]:
def img2rgb(path):
    """
    :param path: String, location of the image file.
    :returns rgb-format numpy array.
    """
    img = cv2.imread(path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

### Load your .ckpt model

In [6]:
model = torch.load(CKPT_PATH)

### Predict the result when an image comes

In [10]:
# convert image to rgb-format numpy array
oneshot = img2rgb(DATA_SAMPLE)
# do dimension transpose: (height, weidth, channel) to (channel, height, weidth)
oneshot = torch.Tensor(oneshot).permute(2,0,1).unsqueeze(0)
# use the cuda wrapper
oneshot = oneshot.cuda()
# input shape
print('shape of oneshot={}'.format(oneshot.shape))
pred = model(oneshot)
# output shape
print('shape of pred={}'.format(pred.shape))
# get a predicted label
_, predicted = torch.max(pred.cpu().data, 1)

shape of oneshot=torch.Size([1, 3, 192, 340])
shape of pred=torch.Size([1, 85])


In [11]:
res = label_df[label_df['label']==predicted.item()]['target'].unique()[0]
print('the predicted result is {}'.format(res))

the predicted result is 通泉草
